In [1]:
import pandas as pd
import glob

In [2]:
import wandb
wandb.login(key = '5c890c5f71b46a63dd49c1b634505dc3159a874b')

wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [3]:
path_dict = {}

In [4]:
for i in path_dict.keys():
    if 'no_damage' in i:
        print("***********" + str(len(path)))

In [5]:
from os import walk
for (dirpath,dir_names,filenames) in walk('../input/satellite-images-of-hurricane-damage'):
    temp = []
    for i in filenames:
        temp.append(dirpath + '/' + i)
    if(len(temp) >0):
        path_dict[dirpath ] = temp

In [6]:
import numpy as np 
import pandas as pd 
import os
import cv2
from tqdm import tqdm 
import glob
import sys

import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2

import pytorch_lightning as pl
import torchmetrics
from pytorch_lightning import loggers as pl_loggers
from pytorch_lightning.loggers import WandbLogger

! pip install timm
import timm

     |████████████████████████████████| 431 kB 928 kB/s            


In [7]:
# img_paths=[]
# img_labels=[]
# for i in glob('../input/satellite-images-of-hurricane-damage/**',recursive=True):
#     if i.endswith('.jpeg'):
#         a=i.split('/')
#         img_paths.append(i)
#         img_labels.append(a[-2])

In [8]:
def do_nothing(image, *args, **kwargs):
    return {"image": image}

def enable_if(condition, obj):
    return obj if (condition) else do_nothing

In [9]:
class Hurricane_Data(Dataset):
    def __init__(self,path_array : list,mode = 'val'):
        self.path_array = path_array 
        
         
        self.compose = A.Compose([
        enable_if(mode == "train", A.Transpose(p=0.5)),
        enable_if(mode == "train", A.HorizontalFlip(p=0.5)),
        enable_if(mode == "train", A.VerticalFlip(p=0.5)),
        enable_if(mode == "train", A.ShiftScaleRotate(p=0.5)),
        enable_if(mode == "train", A.HueSaturationValue(hue_shift_limit=0.2, sat_shift_limit=0.2, val_shift_limit=0.2, p=0.5)),
        enable_if(mode == "train", A.RandomBrightnessContrast(brightness_limit=(-0.1,0.1), contrast_limit=(-0.1, 0.1), p=0.5)),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225], max_pixel_value=255.0, p=1.0),
        enable_if(mode == "train", A.CoarseDropout(p=0.5)),
        enable_if(mode == "train", A.Cutout(p=0.5)),
        ToTensorV2(p=1.0),
    ])
        
        self.compose 
    def __len__(self):
        return len(self.path_array)
    
    def __getitem__(self,index):
        filename = self.path_array[index]
        image = cv2.imread(filename)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = self.compose(image = image)['image']
        if 'no_damage' in filename:
            label = [1,0]
        else:
            label = [0,1]
        
        return torch.tensor(image),torch.tensor(label)

In [10]:
train_damage = '../input/satellite-images-of-hurricane-damage/train_another/damage'
train_no_damage = '../input/satellite-images-of-hurricane-damage/train_another/no_damage'
train_paths = path_dict[train_damage] + path_dict[train_no_damage]

train_damage = '../input/satellite-images-of-hurricane-damage/validation_another/damage'
train_no_damage = '../input/satellite-images-of-hurricane-damage/validation_another/no_damage'
valid_paths = path_dict[train_damage] + path_dict[train_no_damage]

train_damage = '../input/satellite-images-of-hurricane-damage/test_another/damage'
train_no_damage = '../input/satellite-images-of-hurricane-damage/test_another/no_damage'
test_paths = path_dict[train_damage] + path_dict[train_no_damage]


In [11]:
train_paths += valid_paths

In [12]:
class model(nn.Module):
    def __init__(self,model_name = ""):
        super().__init__()
        self.convs = nn.Sequential(
            nn.Conv2d(in_channels = 3,out_channels = 96,kernel_size = 11,stride = 4),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 96),
            nn.MaxPool2d(kernel_size = 3,stride =2),
            
            nn.Conv2d(in_channels = 96,out_channels = 256,kernel_size = 5,stride = 1, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 256),
            nn.MaxPool2d(kernel_size = 3,stride =2),
            
            nn.Conv2d(in_channels = 256,out_channels = 384,kernel_size = 3,stride = 1, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 384),
            nn.Conv2d(in_channels = 384,out_channels = 384,kernel_size = 3,stride = 1, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 384),
            nn.Conv2d(in_channels = 384,out_channels = 256,kernel_size = 3,stride = 1, padding = 'same'),
            nn.ReLU(),
            nn.BatchNorm2d(num_features = 256),
            nn.MaxPool2d(kernel_size = 3,stride =2),
            nn.Flatten()
        )
        
        self.linears = nn.Sequential(
                nn.Linear(in_features = 1024,out_features = 256),
                nn.ReLU(),
                nn.Dropout(p = 0.5),
                nn.Linear(in_features = 256, out_features = 2),
                nn.Sigmoid()
        )
        
    def forward(self,x):
        x = self.convs(x)
        x = self.linears(x)                                  
        return x

In [13]:
class LitHurricane(pl.LightningModule):
    def __init__(self,train_paths : str ,valid_paths : str,test_paths : str):
        super(LitHurricane,self).__init__()
        
        self.train_dataset = Hurricane_Data(train_paths,mode = 'train')
        self.valid_dataset = Hurricane_Data(valid_paths,mode = 'val')
        self.test_dataset = Hurricane_Data(test_paths,mode = 'val')
        
#         self.model = timm.create_model('efficientnet_b0', 
#                                        num_classes=2,
#                                        pretrained = True
#                                       )
        
        self.model = model()
#         self.model = model()
        
        self.train_loss = nn.BCEWithLogitsLoss()
        self.valid_loss = nn.BCEWithLogitsLoss()
        
        self.train_acc = torchmetrics.Accuracy()
        self.valid_acc = torchmetrics.Accuracy()
        
        
        self.lr = 0.001
        
    def forward(self,batch):
        return self.model(batch)
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=1e-4, weight_decay=3e-6)
    
    def train_dataloader(self):
        train_loader = DataLoader(self.train_dataset,
                             batch_size =  32,
                             shuffle = False,
                             sampler = None, 
                                 num_workers = os.cpu_count()
                                 )

        return train_loader
    
    def val_dataloader(self):
        val_loader = DataLoader(self.test_dataset,
                               batch_size = 32,
                               shuffle = False,
                               num_workers = os.cpu_count()
                               )
        return val_loader

    def training_step(self,batch,batch_idx):
        image,labels = batch
        logits = self(image)
        loss = self.train_loss(logits,labels.type_as(logits))
        logits = logits > 0.5
        accuracy = self.train_acc(logits,labels.type_as(logits))
        
        self.log("train_loss_batch", loss,prog_bar = True)
        self.log("train_acc_batch", accuracy,prog_bar = True)
        
        return {
            'loss' : loss,
            'y_pred' : logits,
            'y_true' : labels
        }
    
    
    def training_epoch_end(self,outputs):
        accuracy = self.train_acc.compute()
        self.log("train_acc_end",accuracy,prog_bar = True)
        print(f"Train accuracy for epoch {accuracy}")

    def validation_step(self,batch,batch_idx):
        image,labels = batch
        logits = self.model(image)
        loss = self.valid_loss(logits,labels.float())
        
        logits = (logits > 0.5).int()
        accuracy = self.valid_acc(logits,labels)
        
        self.log("valid_loss_batch", loss,prog_bar = True)
#         self.log("valid_acc_batch", accuracy,prog_bar = True)
        
        return {
            'loss' : loss,
            'y_pred' : logits,
            'y_true' : labels
        }
        
    
    def validation_epoch_end(self,outputs):
        accuracy = self.valid_acc.compute()
        self.log("valid_acc_end",accuracy,prog_bar = True)
        print(f"Valid accuracy for epoch {accuracy}")
        return {
            'val_loss' : outputs[0]['loss'],
            'val_acc_end' : accuracy
        }

In [14]:
from pytorch_lightning.callbacks.progress import ProgressBar

class LitProgressBar(ProgressBar):
    def init_train_tqdm(self):
        bar = super().init_train_tqdm()
        bar.leave = True
        return bar
        
    def init_validation_tqdm(self):
        bar = super().init_validation_tqdm()
        bar.set_description('Valid')
        return bar

        
    def training_epoch_end(self, outputs):
        self.trainer.progress_bar_callback.main_progress_bar.write(
            f"Epoch {self.trainer.current_epoch + 1} training loss={self.trainer.progress_bar_dict['train_loss']}" +
            f"Accuracy={self.trainer.progress_bar_dict['train_acc_end']}"
        )

    def validation_epoch_end(self, outputs):
        loss = torch.stack(outputs).mean()
        self.trainer.progress_bar_callback.main_progress_bar.write(
            f"Epoch {self.trainer.current_epoch + 1} validation loss={self.trainer.progress_bar_dict['val_loss']}" +
            f"Valid Accuracy = {self.trainer.progress_bar_dict['valid_acc_end']}"
        )

In [15]:
import warnings
warnings.filterwarnings('ignore')

In [16]:
wandb_logger = WandbLogger(project="ML_J_COMP")

In [17]:
if __name__ == '__main__':
    lit = LitHurricane(train_paths,valid_paths,test_paths)
    trainer = pl.Trainer(
        logger=wandb_logger,
        max_epochs = 5,
        gradient_clip_val=1,
        progress_bar_refresh_rate = 0,
        callbacks = [LitProgressBar()],
        gpus = 1
    )
    trainer.fit(lit)

Validation sanity check: 0it [00:00, ?it/s]

Valid accuracy for epoch 1.0


Training: 0it [00:00, ?it/s]

wandb: Currently logged in as: gthejin (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.12 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Validating: 0it [00:00, ?it/s]

...

Valid accuracy for epoch 0.25116389989852905
Train accuracy for epoch 0.6552639007568359


Validating: 0it [00:00, ?it/s]

Valid accuracy for epoch 0.3744176924228668
Train accuracy for epoch 0.6393854022026062


Validating: 0it [00:00, ?it/s]

Valid accuracy for epoch 0.3585678040981293
Train accuracy for epoch 0.6481583118438721
